In [1]:
import os.path as osp
import json
import numpy as np

In [2]:
splits = ['miniv', 'val', 'test', 'train']

ref_tasks = []
for s in splits:
    print('loading refer_filtered_instance_%s.json' % s)
    with open('/mnt/nfs/work1/elm/chenyun/PhraseCutPano/data/refvg/amt_result/refer_filtered_instance_%s.json' % s, 'r') as f:
        ref_tasks += json.load(f)


loading refer_filtered_instance_miniv.json
loading refer_filtered_instance_val.json
loading refer_filtered_instance_test.json
loading refer_filtered_instance_train.json


In [3]:
print(ref_tasks[0])

{'phrase_structure': {'attributes': ['seated'], 'type': 'name', 'name': 'hill', 'relations': []}, 'task_id': '2328797__3342390', 'iou': 0.10582814686158151, 'instance_boxes': [[50.505, 259.185, 150.295, 181.375]], 'iop': 1.0, 'image_id': 2328797, 'ann_ids': [3342390], 'Polygons': [[[[67.71000000000001, 305.805], [58.83, 283.605], [57.72, 275.28], [53.834999999999994, 259.185], [50.505, 264.18], [53.28, 283.05], [52.724999999999994, 288.045], [56.055, 314.685], [76.59, 387.945], [78.81, 439.005], [89.91000000000001, 439.56], [192.03, 439.56], [193.14, 346.32], [199.245, 342.435], [199.79999999999998, 338.54999999999995], [79.92, 339.66], [62.714999999999996, 285.825]]]], 'phrase': 'seated hill', 'turk_id': 'A2P7Z2TKGACY2E', 'iob': 0.10582814686158151}


In [4]:
fast_text_path = '/mnt/nfs/work1/elm/chenyun/PhraseCutPano/data/fast_text/'
ft_vocab = np.load(osp.join(fast_text_path, 'vocabulary_ft.npy'))
ft_vocab = list(ft_vocab)
ft_embeddings = np.load(osp.join(fast_text_path, 'embeddings_ft.npy'))

In [5]:
word_count = dict()
special = []


def replace_special(name):
    name = name.lower()
    i = 0
    while i < len(name):
        c = name[i]
        if (c > 'z' or c < 'a') and (c < '0' or c > '9') and c != ' ':
            if c not in special: 
                special.append(c)
            name = name[:i] + ' ' + c + ' ' + name[i+1:]
            i += 2
        i += 1
    return name


for task in ref_tasks:
    phrase = task['phrase']
    phrase = replace_special(phrase)
    words = phrase.split()
    for w in words:
        word_count[w] = word_count.get(w, 0) + 1

In [6]:
print(len(word_count))
print(special)
ws = [w for w, c in word_count.items() if c > 20]
print(len(ws))

7810
['-', "'", ',', ':', '<', '.', '/', '?', '*', '"', '\\', '&', '\x00', '`', '!', ']', '[', '+', '@', '(', ')']
1786


In [7]:
wc_sorted = [(w, c) for w, c in sorted(word_count.items(), key=lambda kv: -kv[1])]

vocab = ['<PAD>', '<UNK>', '<BOS>', '<EOS>'] + [wc[0] for wc in wc_sorted]
freq = [1000] * 4 + [wc[1] for wc in wc_sorted]

print(vocab[:10])
print(freq[:10])

['<PAD>', '<UNK>', '<BOS>', '<EOS>', 'white', 'on', 'black', 'blue', 'green', 'man']
[1000, 1000, 1000, 1000, 25410, 18910, 14959, 11968, 11366, 10976]


In [ ]:
lookup = dict()
lookup['included'] = ['refvg_train', 'refvg_val', 'refvg_test', 'refvg_miniv']
lookup['ix_to_word'] = vocab
lookup['freq'] = freq

embeddings = np.empty((len(vocab), 300))
for i, w in enumerate(vocab):
    if w in ft_vocab:
        ft_i = ft_vocab.index(w)
        embeddings[i] = ft_embeddings[ft_i]
    else:
        embeddings[i] = np.random.randn(300) / 300.0
        print(i, w, freq[i])
lookup['embeddings'] = embeddings

In [9]:
np.save('/mnt/nfs/work1/elm/chenyun/PhraseCutPano/data/fast_text/lookup_refvg_all.npy', lookup)